# 1 tsla-yfinance

In [1]:
#!pip install yfinance
import yfinance as yf
import pandas as pd

tesla_data = yf.Ticker("TSLA")
tesla_data = tesla_data.history(period="max")
tesla_data.reset_index(inplace = True)
tesla_data.dropna(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


# 2 tsla-webscraping

In [2]:
#!pip install requests
#!pip install bs4
#!pip install plotly

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
data  = requests.get(url).text

soup = BeautifulSoup(data, 'html5lib')
tesla_revenue = pd.DataFrame(columns=["Date","Revenue"])

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")    
    tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index = True) 
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.tail()

,Date,Revenue
43,2010-09-30,31
44,2010-06-30,28
45,2010-03-31,21
47,2009-09-30,46
48,2009-06-30,27


# 3 graph tsla stock data

In [8]:
#tesla_data tesla_revenue
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=(
    "Tesla Stock Closing Price Trend", "Tesla Revenue Trend"), vertical_spacing = .3)
fig.add_trace(go.Scatter(x=pd.to_datetime(tesla_data.Date, infer_datetime_format=True), 
                         y=tesla_data.Open.astype("float"), name="Opening Price"), row=1, col=1)
fig.add_trace(go.Scatter(x=pd.to_datetime(tesla_revenue.Date, infer_datetime_format=True), 
                         y=tesla_revenue.Revenue.astype("float"), name="Revenue"), row=2, col=1)
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
fig.update_layout(showlegend=False,height=900,xaxis_rangeslider_visible=True)
fig.show()